In [3]:
### Environment setup
from pystac_client import Client
import planetary_computer as pc
import os

# Set the environment variable PC_SDK_SUBSCRIPTION_KEY, or set it here.
# The Hub sets PC_SDK_SUBSCRIPTION_KEY automatically.
# pc.settings.set_subscription_key(<YOUR API Key>)
env_vars = !cat /content/.env

for var in env_vars:
    key, value = var.split(' = ')
    os.environ[key] = value

In [4]:
import pandas as pd
import fsspec
import numpy as np
import geopandas as gpd
import rasterio
from rasterio import windows
from rasterio import features
from rasterio import warp

import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
#experiments with dask
import dask.dataframe as dd
import dask_geopandas as dg

data_src = 'usgs'
container = 'usgs-data'

storage_options={'account_name':os.environ['ACCOUNT_NAME'],\
                 'account_key':os.environ['BLOB_KEY']}
fs = fsspec.filesystem('az', account_name=storage_options['account_name'], account_key=storage_options['account_key'])
station_url = f'az://{container}/{data_src}_station_metadata_raw.csv'
station_df = pd.read_csv(station_url, storage_options=storage_options)                
#df = dd.read_csv(station_url, storage_options=storage_options)
#ddf = dg.from_dask_dataframe(df).set_geometry(dg.points_from_xy(ddf, 'Latitude', 'Longitude'))

In [51]:
sites_str = [str(f).zfill(8) for f in station_df.site_no]
station_df['sites_str'] = sites_str

In [52]:
query = []
for f in fs.ls(f'{container}/stations'):
    station = os.path.basename(f).split('_')[0]
    query.append(station)
q = pd.DataFrame({'sites_str':query})
out = station_df.merge(q, on='sites_str')
out['site_no'] = out['sites_str']

In [63]:
out = out[['site_no','site_name', 'Latitude', 'Longitude','geometry']]


In [64]:
out.to_csv(f'az://{container}/usgs_station_metadata.csv',index=False, storage_options=storage_options)

In [5]:
url = f'az://{container}/usgs_station_metadata.csv'
z = pd.read_csv(url, storage_options=storage_options)

In [28]:
for f in fs.ls('usgs-data/stations'):
    out = f"{f.split('_')[0]}.csv"
    fs.cp(f, out)